# Power System Optimization Applications

Power system operations rely heavily on optimization to ensure reliable and economical electricity delivery. This lesson explores how optimization techniques are applied to solve critical operational problems that grid operators face every day. We'll implement increasingly sophisticated models that form the foundation of modern electricity markets and system operations.

## Learning Objectives

By the end of this lesson, you will be able to:

1. Implement advanced economic dispatch with real-world complications
2. Formulate and solve the unit commitment problem with mixed-integer programming
3. Incorporate security constraints into optimization models
4. Understand the basics of optimal power flow and its linearizations
5. Handle renewable energy uncertainty in optimization models

Let's begin by importing the necessary libraries for our optimization implementations.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import *
import time

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Advanced Economic Dispatch

While basic economic dispatch assumes simple quadratic cost functions, real generators have more complex cost characteristics. Let's explore how to handle piecewise linear costs, prohibited operating zones, and multi-area dispatch.

### Piecewise Linear Cost Functions

Many generators have cost curves that are better approximated by piecewise linear functions rather than smooth quadratics. This is especially true for units with multiple fuel sources or efficiency variations across their operating range.

In [ ]:
def piecewise_linear_ed(generators, demand):
    """
    Economic dispatch with piecewise linear cost curves.
    
    Each generator has cost segments defined by power ranges and slopes.
    """
    prob = LpProblem("Piecewise_ED", LpMinimize)
    
    # Decision variables for each segment
    p_seg = {}
    for g_id, gen in generators.items():
        for i, seg in enumerate(gen['segments']):
            p_seg[g_id, i] = LpVariable(f"p_{g_id}_{i}", 0, seg['width'])
    
    # Total generation per unit
    p_total = {}
    for g_id in generators:
        p_total[g_id] = lpSum(p_seg[g_id, i] for i in range(len(generators[g_id]['segments'])))
    
    # Objective: minimize total cost
    prob += lpSum(
        generators[g_id]['segments'][i]['slope'] * p_seg[g_id, i]
        for g_id in generators
        for i in range(len(generators[g_id]['segments']))
    )
    
    # Demand constraint
    prob += lpSum(p_total[g_id] for g_id in generators) == demand
    
    # Generator limits
    for g_id, gen in generators.items():
        prob += p_total[g_id] >= gen['p_min']
        prob += p_total[g_id] <= gen['p_max']
    
    prob.solve()
    
    return prob, p_total

# Example with piecewise linear costs
piecewise_generators = {
    'G1': {
        'p_min': 50, 'p_max': 200,
        'segments': [
            {'width': 50, 'slope': 20},  # 50-100 MW
            {'width': 50, 'slope': 22},  # 100-150 MW
            {'width': 50, 'slope': 25}   # 150-200 MW
        ]
    },
    'G2': {
        'p_min': 40, 'p_max': 150,
        'segments': [
            {'width': 60, 'slope': 24},  # 40-100 MW
            {'width': 50, 'slope': 28}   # 100-150 MW
        ]
    }
}

demand = 250
prob, p_total = piecewise_linear_ed(piecewise_generators, demand)

print("Piecewise Linear Economic Dispatch Results:")
print(f"Status: {LpStatus[prob.status]}")
print(f"Total cost: ${value(prob.objective):.2f}")
for g_id in piecewise_generators:
    print(f"{g_id}: {value(p_total[g_id]):.1f} MW")

### Exercise 1: Piecewise Linear Cost with Prohibited Zones

Implement economic dispatch with both piecewise linear costs and prohibited operating zones. Some generators cannot operate in certain ranges due to mechanical vibrations or other technical constraints.

In [ ]:
# Exercise 1: Implement ED with prohibited zones
# Given: 3 generators with prohibited operating zones
# Generator 1: Cannot operate between 75-85 MW
# Generator 2: Cannot operate between 110-120 MW
# Generator 3: No prohibited zones
# Total demand: 300 MW

# Your implementation here

In [ ]:
# Solution
def ed_with_prohibited_zones(demand):
    prob = LpProblem("ED_Prohibited_Zones", LpMinimize)
    
    # Generator data with prohibited zones
    generators = {
        'G1': {'p_min': 50, 'p_max': 150, 'cost': 22, 'prohibited': [(75, 85)]},
        'G2': {'p_min': 40, 'p_max': 180, 'cost': 25, 'prohibited': [(110, 120)]},
        'G3': {'p_min': 30, 'p_max': 120, 'cost': 30, 'prohibited': []}
    }
    
    # Decision variables for each operating zone
    p_zones = {}
    z_zones = {}  # Binary variables for zone selection
    
    for g_id, gen in generators.items():
        if not gen['prohibited']:
            # No prohibited zones
            p_zones[g_id, 0] = LpVariable(f"p_{g_id}_0", gen['p_min'], gen['p_max'])
        else:
            # Create zones around prohibited regions
            zones = []
            start = gen['p_min']
            
            for prohib_start, prohib_end in gen['prohibited']:
                zones.append((start, prohib_start))
                start = prohib_end
            zones.append((start, gen['p_max']))
            
            for i, (z_min, z_max) in enumerate(zones):
                p_zones[g_id, i] = LpVariable(f"p_{g_id}_{i}", 0)
                z_zones[g_id, i] = LpVariable(f"z_{g_id}_{i}", cat='Binary')
                
                # Zone constraints
                prob += p_zones[g_id, i] >= z_min * z_zones[g_id, i]
                prob += p_zones[g_id, i] <= z_max * z_zones[g_id, i]
            
            # Only one zone active
            prob += lpSum(z_zones[g_id, i] for i in range(len(zones))) == 1
    
    # Total generation per unit
    p_total = {}
    for g_id in generators:
        if not generators[g_id]['prohibited']:
            p_total[g_id] = p_zones[g_id, 0]
        else:
            n_zones = len([k for k in p_zones if k[0] == g_id])
            p_total[g_id] = lpSum(p_zones[g_id, i] for i in range(n_zones))
    
    # Objective
    prob += lpSum(generators[g_id]['cost'] * p_total[g_id] for g_id in generators)
    
    # Demand constraint
    prob += lpSum(p_total[g_id] for g_id in generators) == demand
    
    prob.solve()
    
    print("\nEconomic Dispatch with Prohibited Zones:")
    print(f"Status: {LpStatus[prob.status]}")
    print(f"Total cost: ${value(prob.objective):.2f}")
    for g_id in generators:
        gen_output = value(p_total[g_id])
        print(f"{g_id}: {gen_output:.1f} MW", end="")
        
        # Check if in prohibited zone
        for p_start, p_end in generators[g_id]['prohibited']:
            if p_start <= gen_output <= p_end:
                print(" (ERROR: In prohibited zone!)", end="")
        print()
    
    return prob, p_total

# Run the solution
prob, p_total = ed_with_prohibited_zones(300)

# Visualize the solution
fig, ax = plt.subplots(figsize=(12, 6))
for i, (g_id, gen) in enumerate({'G1': {'p_min': 50, 'p_max': 150, 'prohibited': [(75, 85)]},
                                 'G2': {'p_min': 40, 'p_max': 180, 'prohibited': [(110, 120)]},
                                 'G3': {'p_min': 30, 'p_max': 120, 'prohibited': []}}.items()):
    y_pos = i
    
    # Operating range
    ax.barh(y_pos, gen['p_max'] - gen['p_min'], left=gen['p_min'], 
            height=0.3, color='lightblue', alpha=0.5, label='Operating range' if i==0 else '')
    
    # Prohibited zones
    for p_start, p_end in gen['prohibited']:
        ax.barh(y_pos, p_end - p_start, left=p_start, 
                height=0.3, color='red', alpha=0.7, label='Prohibited zone' if i==0 else '')
    
    # Actual generation
    gen_value = value(p_total[g_id])
    ax.scatter(gen_value, y_pos, s=200, color='green', zorder=3, 
               label='Dispatch point' if i==0 else '')
    ax.text(gen_value, y_pos + 0.2, f'{gen_value:.1f} MW', ha='center')

ax.set_yticks(range(3))
ax.set_yticklabels(['G1', 'G2', 'G3'])
ax.set_xlabel('Power (MW)')
ax.set_title('Economic Dispatch with Prohibited Operating Zones')
ax.legend()
plt.tight_layout()
plt.show()

## 2. Unit Commitment Problem

Unit commitment (UC) determines which generators to turn on or off over a planning horizon (typically 24-48 hours) while respecting technical constraints. This is a mixed-integer programming problem that balances operating costs against startup costs.

In [ ]:
# Unit commitment implementation
def create_uc_data():
    """Create realistic unit commitment test data."""
    # Generator data
    generators = pd.DataFrame({
        'p_min': [100, 80, 50, 40, 20],
        'p_max': [400, 300, 200, 150, 100],
        'cost': [22, 25, 30, 28, 35],
        'startup_cost': [5000, 4000, 2000, 1500, 1000],
        'min_up': [8, 6, 4, 3, 2],
        'min_down': [8, 6, 4, 3, 2],
        'ramp_up': [80, 60, 50, 40, 30],
        'ramp_down': [80, 60, 50, 40, 30],
        'initial_status': [1, 1, 0, 0, 0],
        'initial_power': [200, 150, 0, 0, 0]
    }, index=['G1', 'G2', 'G3', 'G4', 'G5'])
    
    # 24-hour demand profile
    hours = np.arange(24)
    base_demand = 350
    demand = base_demand + 150 * np.sin((hours - 6) * np.pi / 12) + \
             50 * np.sin(2 * hours * np.pi / 12) + \
             np.random.normal(0, 10, 24)
    demand = np.maximum(demand, 250)  # Minimum demand
    
    return generators, demand

generators, demand = create_uc_data()

# Visualize demand profile
plt.figure(figsize=(12, 5))
plt.bar(range(24), demand, color='skyblue', edgecolor='navy', alpha=0.7)
plt.plot(range(24), demand, 'r-', linewidth=2)
plt.xlabel('Hour')
plt.ylabel('Demand (MW)')
plt.title('24-Hour Demand Profile')
plt.grid(True, alpha=0.3)
plt.show()

print("Generator Characteristics:")
print(generators)

### Exercise 2: Complete Unit Commitment Implementation

Implement the full unit commitment problem with all constraints including minimum up/down times, ramping limits, and reserve requirements. This is the core optimization problem that runs in every ISO/RTO market.

In [ ]:
# Exercise 2: Implement complete unit commitment
# Requirements:
# - Binary on/off decisions for each generator
# - Minimum up/down time constraints
# - Ramping constraints
# - 10% spinning reserve requirement
# - Minimize total cost (operating + startup)

# Your implementation here

In [ ]:
# Solution
def unit_commitment(generators, demand, reserve_pct=0.1):
    """Solve unit commitment problem with all constraints."""
    T = len(demand)
    G = len(generators)
    periods = range(T)
    gen_names = generators.index.tolist()
    
    # Create problem
    prob = LpProblem("Unit_Commitment", LpMinimize)
    
    # Decision variables
    u = {}  # Unit on/off
    p = {}  # Power output
    v = {}  # Startup indicator
    w = {}  # Shutdown indicator
    
    for g in gen_names:
        for t in periods:
            u[g,t] = LpVariable(f"u_{g}_{t}", cat='Binary')
            p[g,t] = LpVariable(f"p_{g}_{t}", 0)
            v[g,t] = LpVariable(f"v_{g}_{t}", cat='Binary')
            w[g,t] = LpVariable(f"w_{g}_{t}", cat='Binary')
    
    # Objective: minimize total cost
    prob += lpSum(
        generators.loc[g, 'cost'] * p[g,t] + 
        generators.loc[g, 'startup_cost'] * v[g,t]
        for g in gen_names for t in periods
    )
    
    # Constraints
    for t in periods:
        # Demand constraint
        prob += lpSum(p[g,t] for g in gen_names) == demand[t], f"Demand_{t}"
        
        # Reserve constraint
        prob += lpSum(
            generators.loc[g, 'p_max'] * u[g,t] - p[g,t] 
            for g in gen_names
        ) >= reserve_pct * demand[t], f"Reserve_{t}"
    
    for g in gen_names:
        for t in periods:
            # Generation limits
            prob += p[g,t] >= generators.loc[g, 'p_min'] * u[g,t], f"P_min_{g}_{t}"
            prob += p[g,t] <= generators.loc[g, 'p_max'] * u[g,t], f"P_max_{g}_{t}"
            
            # Startup/shutdown logic
            if t == 0:
                # Initial conditions
                initial_u = generators.loc[g, 'initial_status']
                prob += v[g,t] >= u[g,t] - initial_u, f"Startup_{g}_{t}"
                prob += w[g,t] >= initial_u - u[g,t], f"Shutdown_{g}_{t}"
            else:
                prob += v[g,t] >= u[g,t] - u[g,t-1], f"Startup_{g}_{t}"
                prob += w[g,t] >= u[g,t-1] - u[g,t], f"Shutdown_{g}_{t}"
            
            # Ramping constraints
            if t > 0:
                # Ramp up
                prob += (p[g,t] - p[g,t-1] <= 
                        generators.loc[g, 'ramp_up'] * u[g,t-1] + 
                        generators.loc[g, 'p_max'] * v[g,t]), f"RampUp_{g}_{t}"
                # Ramp down
                prob += (p[g,t-1] - p[g,t] <= 
                        generators.loc[g, 'ramp_down'] * u[g,t] + 
                        generators.loc[g, 'p_max'] * w[g,t]), f"RampDown_{g}_{t}"
            else:
                # Initial ramping
                initial_p = generators.loc[g, 'initial_power']
                prob += (p[g,t] - initial_p <= 
                        generators.loc[g, 'ramp_up'] * generators.loc[g, 'initial_status'] + 
                        generators.loc[g, 'p_max'] * v[g,t]), f"RampUp_{g}_{t}"
    
    # Minimum up/down time constraints
    for g in gen_names:
        min_up = int(generators.loc[g, 'min_up'])
        min_down = int(generators.loc[g, 'min_down'])
        
        # Minimum up time
        for t in periods:
            t_end = min(t + min_up, T)
            if t_end > t:
                prob += (lpSum(u[g,k] for k in range(t, t_end)) >= 
                        min_up * v[g,t]), f"MinUp_{g}_{t}"
        
        # Minimum down time
        for t in periods:
            t_end = min(t + min_down, T)
            if t_end > t:
                prob += (lpSum(1 - u[g,k] for k in range(t, t_end)) >= 
                        min_down * w[g,t]), f"MinDown_{g}_{t}"
    
    # Solve
    start_time = time.time()
    prob.solve(PULP_CBC_CMD(timeLimit=60, msg=0))
    solve_time = time.time() - start_time
    
    return prob, u, p, v, w, solve_time

# Solve unit commitment
prob, u, p, v, w, solve_time = unit_commitment(generators, demand)

print(f"\nUnit Commitment Results:")
print(f"Status: {LpStatus[prob.status]}")
print(f"Total cost: ${value(prob.objective):,.0f}")
print(f"Solve time: {solve_time:.2f} seconds")

# Create commitment schedule
schedule = pd.DataFrame(index=generators.index, columns=range(24))
generation = pd.DataFrame(index=generators.index, columns=range(24))

for g in generators.index:
    for t in range(24):
        schedule.loc[g, t] = int(value(u[g,t]) > 0.5)
        generation.loc[g, t] = value(p[g,t]) if schedule.loc[g, t] else 0

# Visualize results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Unit commitment schedule
colors = plt.cm.Set3(np.linspace(0, 1, len(generators)))
bottom = np.zeros(24)

for i, g in enumerate(generators.index):
    gen_output = generation.loc[g].values
    ax1.bar(range(24), gen_output, bottom=bottom, label=g, 
            color=colors[i], edgecolor='black', linewidth=0.5)
    bottom += gen_output

ax1.plot(range(24), demand, 'k--', linewidth=2, label='Demand')
ax1.set_xlabel('Hour')
ax1.set_ylabel('Power (MW)')
ax1.set_title('Unit Commitment Schedule and Generation')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# On/off status
im = ax2.imshow(schedule.values, cmap='RdYlGn', aspect='auto')
ax2.set_yticks(range(len(generators)))
ax2.set_yticklabels(generators.index)
ax2.set_xlabel('Hour')
ax2.set_title('Unit On/Off Status (Green=On, Red=Off)')
ax2.set_xticks(range(0, 24, 2))

# Add startup markers
for i, g in enumerate(generators.index):
    for t in range(24):
        if value(v[g,t]) > 0.5:
            ax2.text(t, i, 'START', ha='center', va='center', 
                    fontsize=8, fontweight='bold')

plt.tight_layout()
plt.show()

# Summary statistics
total_startup_cost = sum(generators.loc[g, 'startup_cost'] * value(v[g,t]) 
                         for g in generators.index for t in range(24))
total_operating_cost = value(prob.objective) - total_startup_cost

print(f"\nCost Breakdown:")
print(f"Operating cost: ${total_operating_cost:,.0f}")
print(f"Startup cost: ${total_startup_cost:,.0f}")
print(f"\nStartup events:")
for g in generators.index:
    startups = [t for t in range(24) if value(v[g,t]) > 0.5]
    if startups:
        print(f"{g}: hours {startups}")

## 3. Security-Constrained Optimization

Power systems must maintain reliability even when equipment fails. Security-constrained optimization ensures the system can withstand contingencies (N-1 criterion) without violating operational limits.

In [ ]:
def create_network_data():
    """Create a simple 3-bus test system."""
    buses = pd.DataFrame({
        'demand': [0, 150, 100]
    }, index=['Bus1', 'Bus2', 'Bus3'])
    
    generators = pd.DataFrame({
        'bus': ['Bus1', 'Bus1', 'Bus3'],
        'p_max': [150, 100, 120],
        'cost': [20, 25, 22]
    }, index=['G1', 'G2', 'G3'])
    
    lines = pd.DataFrame({
        'from_bus': ['Bus1', 'Bus1', 'Bus2'],
        'to_bus': ['Bus2', 'Bus3', 'Bus3'],
        'limit': [100, 80, 60],
        'reactance': [0.1, 0.15, 0.2]
    }, index=['L1', 'L2', 'L3'])
    
    # Calculate PTDF matrix (simplified DC power flow)
    n_buses = len(buses)
    n_lines = len(lines)
    
    # Build B matrix (excluding slack bus)
    B = np.zeros((n_buses-1, n_buses-1))
    bus_names = buses.index.tolist()
    
    for _, line in lines.iterrows():
        i = bus_names.index(line['from_bus'])
        j = bus_names.index(line['to_bus'])
        y = 1 / line['reactance']
        
        if i > 0 and j > 0:  # Both non-slack
            B[i-1, i-1] += y
            B[j-1, j-1] += y
            B[i-1, j-1] -= y
            B[j-1, i-1] -= y
        elif i > 0:  # Only i is non-slack
            B[i-1, i-1] += y
        elif j > 0:  # Only j is non-slack
            B[j-1, j-1] += y
    
    B_inv = np.linalg.inv(B)
    
    # Calculate PTDF
    PTDF = np.zeros((n_lines, n_buses))
    
    for l, line in lines.iterrows():
        i = bus_names.index(line['from_bus'])
        j = bus_names.index(line['to_bus'])
        y = 1 / line['reactance']
        
        for k in range(n_buses):
            if k == 0:  # Slack bus
                continue
            
            theta_i = 0 if i == 0 else B_inv[i-1, k-1]
            theta_j = 0 if j == 0 else B_inv[j-1, k-1]
            PTDF[l, k] = y * (theta_i - theta_j)
    
    return buses, generators, lines, PTDF

buses, gens, lines, PTDF = create_network_data()
print("Test System Data:")
print("\nGenerators:")
print(gens)
print("\nTransmission Lines:")
print(lines)
print("\nPTDF Matrix:")
print(pd.DataFrame(PTDF, index=lines.index, columns=buses.index).round(3))

### Exercise 3: Security-Constrained Economic Dispatch

Implement security-constrained economic dispatch (SCED) that ensures all line flows remain within limits both in normal conditions and after any single line outage (N-1 contingencies).

In [ ]:
# Exercise 3: Implement SCED with N-1 contingencies
# Requirements:
# - Minimize generation cost
# - Respect line flow limits in base case
# - Respect line flow limits after any single line outage
# - Use linear sensitivity factors (PTDF/LODF)

# Your implementation here

In [ ]:
# Solution
def security_constrained_ed(buses, generators, lines, PTDF):
    """Solve SCED with N-1 contingency constraints."""
    prob = LpProblem("SCED", LpMinimize)
    
    # Decision variables
    p_gen = {}
    for g in generators.index:
        p_gen[g] = LpVariable(f"p_{g}", 0, generators.loc[g, 'p_max'])
    
    # Net injection at each bus
    p_inj = {}
    for b in buses.index:
        gen_at_bus = generators[generators['bus'] == b].index
        p_inj[b] = lpSum(p_gen[g] for g in gen_at_bus) - buses.loc[b, 'demand']
    
    # Objective
    prob += lpSum(generators.loc[g, 'cost'] * p_gen[g] for g in generators.index)
    
    # Power balance
    prob += lpSum(p_inj[b] for b in buses.index) == 0, "Power_Balance"
    
    # Base case line flow constraints
    flows_base = {}
    for l in lines.index:
        flows_base[l] = lpSum(PTDF[lines.index.get_loc(l), buses.index.get_loc(b)] * p_inj[b] 
                             for b in buses.index)
        prob += flows_base[l] <= lines.loc[l, 'limit'], f"Flow_limit_pos_{l}"
        prob += flows_base[l] >= -lines.loc[l, 'limit'], f"Flow_limit_neg_{l}"
    
    # Calculate LODF (Line Outage Distribution Factors)
    n_lines = len(lines)
    LODF = np.zeros((n_lines, n_lines))
    
    for k in range(n_lines):  # Outaged line
        for l in range(n_lines):  # Monitored line
            if k != l:
                # LODF[l,k] = change in flow on line l when line k is out
                # Simplified calculation
                LODF[l, k] = 0.3  # Placeholder - real calculation is more complex
    
    # N-1 contingency constraints
    for k in lines.index:  # Outaged line
        k_idx = lines.index.get_loc(k)
        for l in lines.index:  # Monitored line
            if l != k:
                l_idx = lines.index.get_loc(l)
                # Post-contingency flow
                flow_cont = flows_base[l] + LODF[l_idx, k_idx] * flows_base[k]
                
                prob += flow_cont <= lines.loc[l, 'limit'], f"Cont_flow_pos_{l}_out_{k}"
                prob += flow_cont >= -lines.loc[l, 'limit'], f"Cont_flow_neg_{l}_out_{k}"
    
    # Solve
    prob.solve()
    
    return prob, p_gen, flows_base

# Solve SCED
prob, p_gen, flows = security_constrained_ed(buses, gens, lines, PTDF)

print("\nSecurity-Constrained Economic Dispatch Results:")
print(f"Status: {LpStatus[prob.status]}")
print(f"Total cost: ${value(prob.objective):.2f}")

print("\nGenerator Dispatch:")
for g in gens.index:
    print(f"{g}: {value(p_gen[g]):.1f} MW at {gens.loc[g, 'bus']}")

print("\nLine Flows (Base Case):")
for l in lines.index:
    flow = value(flows[l])
    limit = lines.loc[l, 'limit']
    utilization = abs(flow) / limit * 100
    print(f"{l}: {flow:6.1f} MW / {limit:3.0f} MW ({utilization:4.1f}% utilized)")

# Visualize the network
fig, ax = plt.subplots(figsize=(10, 8))

# Bus positions
pos = {'Bus1': (0, 1), 'Bus2': (1, 0), 'Bus3': (1, 2)}

# Draw buses
for bus, (x, y) in pos.items():
    circle = plt.Circle((x, y), 0.15, color='lightblue', ec='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, y, bus, ha='center', va='center', fontweight='bold')
    
    # Add demand
    demand = buses.loc[bus, 'demand']
    if demand > 0:
        ax.text(x, y-0.25, f"D: {demand} MW", ha='center', fontsize=9)
    
    # Add generation
    gen_at_bus = gens[gens['bus'] == bus]
    if len(gen_at_bus) > 0:
        total_gen = sum(value(p_gen[g]) for g in gen_at_bus.index)
        ax.text(x, y+0.25, f"G: {total_gen:.1f} MW", ha='center', fontsize=9, color='green')

# Draw lines with flows
for _, line in lines.iterrows():
    x1, y1 = pos[line['from_bus']]
    x2, y2 = pos[line['to_bus']]
    
    # Line
    ax.plot([x1, x2], [y1, y2], 'k-', linewidth=2)
    
    # Flow arrow and value
    flow = value(flows[line.name])
    mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
    
    if abs(flow) > 0.1:
        dx, dy = x2 - x1, y2 - y1
        arrow_scale = 0.1 if flow > 0 else -0.1
        ax.arrow(mid_x - dx*arrow_scale, mid_y - dy*arrow_scale, 
                dx*arrow_scale*2, dy*arrow_scale*2,
                head_width=0.05, head_length=0.05, fc='red', ec='red')
    
    ax.text(mid_x + 0.1, mid_y + 0.1, f"{flow:.1f} MW\n({abs(flow)/line['limit']*100:.0f}%)", 
            ha='center', fontsize=9, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-0.5, 2.5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Security-Constrained Economic Dispatch Solution', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Optimal Power Flow Basics

Optimal Power Flow (OPF) extends economic dispatch to include the full AC power flow equations, voltage constraints, and reactive power. While the full AC OPF is nonlinear and computationally challenging, various linearizations make it tractable for large systems.

### Exercise 4: DC OPF vs Linearized AC OPF

Compare DC OPF (ignoring losses and reactive power) with a linearized AC OPF that approximates losses and includes voltage angle constraints.

In [ ]:
# Exercise 4: Implement and compare DC OPF with linearized AC OPF
# Requirements:
# - DC OPF: standard formulation
# - Linearized AC OPF: include loss approximation
# - Compare: total cost, generation dispatch, losses
# - Voltage angle limits: ±30 degrees

# Your implementation here

In [ ]:
# Solution
def dc_opf(buses, generators, lines, PTDF):
    """Standard DC OPF formulation."""
    prob = LpProblem("DC_OPF", LpMinimize)
    
    # Variables
    p_gen = {g: LpVariable(f"p_{g}", 0, generators.loc[g, 'p_max']) 
             for g in generators.index}
    theta = {b: LpVariable(f"theta_{b}", -30*np.pi/180, 30*np.pi/180) 
             for b in buses.index}
    
    # Fix slack bus angle
    prob += theta['Bus1'] == 0
    
    # Objective
    prob += lpSum(generators.loc[g, 'cost'] * p_gen[g] for g in generators.index)
    
    # Power balance at each bus
    for b in buses.index:
        gen_at_bus = generators[generators['bus'] == b].index
        
        # Power balance using DC power flow
        power_injection = lpSum(p_gen[g] for g in gen_at_bus) - buses.loc[b, 'demand']
        
        # DC power flow equations
        flow_out = 0
        for _, line in lines.iterrows():
            if line['from_bus'] == b:
                flow_out += (theta[b] - theta[line['to_bus']]) / line['reactance']
            elif line['to_bus'] == b:
                flow_out += (theta[b] - theta[line['from_bus']]) / line['reactance']
        
        prob += power_injection == flow_out, f"PowerBalance_{b}"
    
    # Line flow limits
    for _, line in lines.iterrows():
        flow = (theta[line['from_bus']] - theta[line['to_bus']]) / line['reactance']
        prob += flow <= line['limit'], f"FlowLimit_pos_{line.name}"
        prob += flow >= -line['limit'], f"FlowLimit_neg_{line.name}"
    
    prob.solve()
    return prob, p_gen, theta

def linearized_ac_opf(buses, generators, lines, PTDF):
    """Linearized AC OPF with loss approximation."""
    prob = LpProblem("Linearized_AC_OPF", LpMinimize)
    
    # Variables
    p_gen = {g: LpVariable(f"p_{g}", 0, generators.loc[g, 'p_max']) 
             for g in generators.index}
    theta = {b: LpVariable(f"theta_{b}", -30*np.pi/180, 30*np.pi/180) 
             for b in buses.index}
    p_loss = LpVariable("p_loss", 0)  # Total system losses
    
    # Fix slack bus
    prob += theta['Bus1'] == 0
    
    # Objective (include cost of losses)
    avg_cost = generators['cost'].mean()
    prob += lpSum(generators.loc[g, 'cost'] * p_gen[g] for g in generators.index) + avg_cost * p_loss
    
    # Loss approximation (simplified - proportional to flow squared)
    # Using piecewise linear approximation
    loss_segments = 5
    loss_flows = {}
    loss_vars = {}
    
    for _, line in lines.iterrows():
        flow = (theta[line['from_bus']] - theta[line['to_bus']]) / line['reactance']
        loss_flows[line.name] = flow
        
        # Piecewise linear approximation of flow^2 * resistance
        resistance = line['reactance'] * 0.1  # Approximate R = 0.1 * X
        for i in range(loss_segments):
            loss_vars[line.name, i] = LpVariable(f"loss_{line.name}_{i}", 0)
    
    # Approximate total losses
    prob += p_loss == lpSum(loss_vars[l, i] for l in lines.index for i in range(loss_segments)) * 0.01
    
    # Power balance with losses
    total_gen = lpSum(p_gen[g] for g in generators.index)
    total_demand = buses['demand'].sum()
    prob += total_gen == total_demand + p_loss, "PowerBalance_with_losses"
    
    # Nodal power balance
    for b in buses.index:
        gen_at_bus = generators[generators['bus'] == b].index
        power_injection = lpSum(p_gen[g] for g in gen_at_bus) - buses.loc[b, 'demand']
        
        flow_out = 0
        for _, line in lines.iterrows():
            if line['from_bus'] == b:
                flow_out += (theta[b] - theta[line['to_bus']]) / line['reactance']
            elif line['to_bus'] == b:
                flow_out += (theta[b] - theta[line['from_bus']]) / line['reactance']
        
        if b != 'Bus1':  # Slack bus absorbs losses
            prob += power_injection == flow_out, f"NodeBalance_{b}"
    
    # Line limits
    for _, line in lines.iterrows():
        prob += loss_flows[line.name] <= line['limit']
        prob += loss_flows[line.name] >= -line['limit']
    
    prob.solve()
    return prob, p_gen, theta, p_loss

# Solve both formulations
print("Solving DC OPF...")
dc_prob, dc_p_gen, dc_theta = dc_opf(buses, gens, lines, PTDF)

print("\nSolving Linearized AC OPF...")
ac_prob, ac_p_gen, ac_theta, ac_p_loss = linearized_ac_opf(buses, gens, lines, PTDF)

# Compare results
print("\n" + "="*60)
print("COMPARISON: DC OPF vs Linearized AC OPF")
print("="*60)

print("\nTotal Cost:")
print(f"DC OPF: ${value(dc_prob.objective):.2f}")
print(f"AC OPF: ${value(ac_prob.objective):.2f}")
print(f"Difference: ${value(ac_prob.objective) - value(dc_prob.objective):.2f}")

print("\nGeneration Dispatch:")
comparison_data = []
for g in gens.index:
    dc_gen = value(dc_p_gen[g])
    ac_gen = value(ac_p_gen[g])
    comparison_data.append({
        'Generator': g,
        'DC OPF (MW)': dc_gen,
        'AC OPF (MW)': ac_gen,
        'Difference (MW)': ac_gen - dc_gen
    })

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

print(f"\nSystem Losses:")
print(f"DC OPF: 0.0 MW (ignored)")
print(f"AC OPF: {value(ac_p_loss):.2f} MW")

print("\nVoltage Angles (degrees):")
for b in buses.index:
    dc_angle = value(dc_theta[b]) * 180 / np.pi
    ac_angle = value(ac_theta[b]) * 180 / np.pi
    print(f"{b}: DC={dc_angle:6.2f}°, AC={ac_angle:6.2f}°")

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Generation comparison
x = np.arange(len(gens))
width = 0.35

dc_values = [value(dc_p_gen[g]) for g in gens.index]
ac_values = [value(ac_p_gen[g]) for g in gens.index]

ax1.bar(x - width/2, dc_values, width, label='DC OPF', color='skyblue')
ax1.bar(x + width/2, ac_values, width, label='AC OPF', color='orange')
ax1.set_xlabel('Generator')
ax1.set_ylabel('Power Output (MW)')
ax1.set_title('Generation Dispatch Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(gens.index)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cost breakdown
dc_cost = value(dc_prob.objective)
ac_gen_cost = sum(gens.loc[g, 'cost'] * value(ac_p_gen[g]) for g in gens.index)
ac_loss_cost = value(ac_prob.objective) - ac_gen_cost

ax2.bar(['DC OPF\n(Gen only)', 'AC OPF\n(Gen)', 'AC OPF\n(Losses)'], 
        [dc_cost, ac_gen_cost, ac_loss_cost],
        color=['skyblue', 'orange', 'red'])
ax2.set_ylabel('Cost ($)')
ax2.set_title('Cost Breakdown')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Renewable Integration Challenges

The increasing penetration of renewable energy introduces uncertainty into power system operations. We need optimization formulations that can handle this variability while maintaining reliability.

### Exercise 5: Unit Commitment with Wind Uncertainty

Modify the unit commitment formulation to include wind generation with uncertainty. Use a simple scenario-based approach to handle multiple possible wind output levels.

In [ ]:
# Exercise 5: UC with stochastic wind generation
# Requirements:
# - Include a 100 MW wind farm
# - Consider 3 scenarios: Low (20%), Medium (50%), High (80%) capacity factor
# - Scenario probabilities: 0.3, 0.5, 0.2
# - Minimize expected cost across scenarios
# - First-stage: unit commitment decisions
# - Second-stage: generation dispatch per scenario

# Your implementation here

In [ ]:
# Solution
def uc_with_wind(generators, demand, wind_capacity=100):
    """Unit commitment with stochastic wind generation."""
    T = len(demand)
    periods = range(T)
    gen_names = generators.index.tolist()
    
    # Wind scenarios
    scenarios = {
        'Low': {'cf': 0.2, 'prob': 0.3},
        'Medium': {'cf': 0.5, 'prob': 0.5},
        'High': {'cf': 0.8, 'prob': 0.2}
    }
    
    # Generate wind profiles for each scenario
    wind_profiles = {}
    for s, data in scenarios.items():
        base_cf = data['cf']
        # Add hourly variation
        hourly_cf = base_cf + 0.1 * np.sin(2 * np.pi * np.arange(T) / T)
        hourly_cf = np.clip(hourly_cf, 0, 1)
        wind_profiles[s] = wind_capacity * hourly_cf
    
    # Create problem
    prob = LpProblem("UC_Stochastic_Wind", LpMinimize)
    
    # First-stage variables (unit commitment)
    u = {}  # Unit on/off
    v = {}  # Startup
    w = {}  # Shutdown
    
    for g in gen_names:
        for t in periods:
            u[g,t] = LpVariable(f"u_{g}_{t}", cat='Binary')
            v[g,t] = LpVariable(f"v_{g}_{t}", cat='Binary')
            w[g,t] = LpVariable(f"w_{g}_{t}", cat='Binary')
    
    # Second-stage variables (dispatch per scenario)
    p = {}  # Power output
    wind_used = {}  # Wind power used
    
    for s in scenarios:
        for g in gen_names:
            for t in periods:
                p[s,g,t] = LpVariable(f"p_{s}_{g}_{t}", 0)
        for t in periods:
            wind_used[s,t] = LpVariable(f"wind_{s}_{t}", 0, wind_profiles[s][t])
    
    # Objective: expected cost
    startup_cost = lpSum(generators.loc[g, 'startup_cost'] * v[g,t] 
                        for g in gen_names for t in periods)
    
    expected_gen_cost = lpSum(
        scenarios[s]['prob'] * generators.loc[g, 'cost'] * p[s,g,t]
        for s in scenarios for g in gen_names for t in periods
    )
    
    prob += startup_cost + expected_gen_cost
    
    # First-stage constraints (commitment)
    for g in gen_names:
        for t in periods:
            # Startup/shutdown logic
            if t == 0:
                initial_u = generators.loc[g, 'initial_status']
                prob += v[g,t] >= u[g,t] - initial_u
                prob += w[g,t] >= initial_u - u[g,t]
            else:
                prob += v[g,t] >= u[g,t] - u[g,t-1]
                prob += w[g,t] >= u[g,t-1] - u[g,t]
    
    # Minimum up/down times (simplified)
    for g in gen_names:
        min_up = int(generators.loc[g, 'min_up'])
        for t in range(T - min_up + 1):
            prob += lpSum(u[g,k] for k in range(t, t + min_up)) >= min_up * v[g,t]
    
    # Second-stage constraints (dispatch per scenario)
    for s in scenarios:
        for t in periods:
            # Demand balance
            prob += (lpSum(p[s,g,t] for g in gen_names) + wind_used[s,t] == 
                    demand[t], f"Demand_{s}_{t}")
            
            # Reserve (additional due to wind uncertainty)
            reserve_req = 0.1 * demand[t] + 0.2 * wind_used[s,t]
            prob += (lpSum(generators.loc[g, 'p_max'] * u[g,t] - p[s,g,t] 
                          for g in gen_names) >= reserve_req, f"Reserve_{s}_{t}")
        
        for g in gen_names:
            for t in periods:
                # Generation limits
                prob += p[s,g,t] >= generators.loc[g, 'p_min'] * u[g,t]
                prob += p[s,g,t] <= generators.loc[g, 'p_max'] * u[g,t]
                
                # Ramping (simplified)
                if t > 0:
                    prob += (p[s,g,t] - p[s,g,t-1] <= 
                            generators.loc[g, 'ramp_up'])
                    prob += (p[s,g,t-1] - p[s,g,t] <= 
                            generators.loc[g, 'ramp_down'])
    
    # Solve
    prob.solve(PULP_CBC_CMD(timeLimit=120, msg=0))
    
    return prob, u, p, wind_used, wind_profiles

# Solve stochastic UC
print("Solving stochastic unit commitment with wind...")
prob, u, p, wind_used, wind_profiles = uc_with_wind(generators, demand)

print(f"\nStochastic UC Results:")
print(f"Status: {LpStatus[prob.status]}")
print(f"Expected total cost: ${value(prob.objective):,.0f}")

# Analyze results
scenarios = {'Low': 0.3, 'Medium': 0.5, 'High': 0.2}

# Create visualization
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for idx, (s, prob_s) in enumerate(scenarios.items()):
    ax = axes[idx]
    
    # Stack plot for each scenario
    bottom = np.zeros(24)
    colors = plt.cm.Set3(np.linspace(0, 1, len(generators) + 1))
    
    # Thermal generation
    for i, g in enumerate(generators.index):
        gen_output = [value(p[s,g,t]) for t in range(24)]
        ax.bar(range(24), gen_output, bottom=bottom, label=g if idx==0 else "",
               color=colors[i], edgecolor='black', linewidth=0.5)
        bottom += gen_output
    
    # Wind generation
    wind_output = [value(wind_used[s,t]) for t in range(24)]
    ax.bar(range(24), wind_output, bottom=bottom, label='Wind' if idx==0 else "",
           color='lightgreen', edgecolor='black', linewidth=0.5)
    
    # Available wind
    ax.plot(range(24), wind_profiles[s], 'g--', linewidth=2, 
            label='Available Wind' if idx==0 else "")
    
    # Demand
    ax.plot(range(24), demand, 'k-', linewidth=2, label='Demand' if idx==0 else "")
    
    ax.set_ylabel('Power (MW)')
    ax.set_title(f'{s} Wind Scenario (Probability: {prob_s})')
    ax.grid(True, alpha=0.3)
    
    if idx == 0:
        ax.legend(loc='upper right')
    if idx == 2:
        ax.set_xlabel('Hour')

plt.tight_layout()
plt.show()

# Wind utilization analysis
print("\nWind Utilization by Scenario:")
for s in scenarios:
    total_available = sum(wind_profiles[s])
    total_used = sum(value(wind_used[s,t]) for t in range(24))
    utilization = total_used / total_available * 100
    print(f"{s}: {total_used:.1f} / {total_available:.1f} MWh ({utilization:.1f}%)")

# Cost breakdown
startup_costs = sum(generators.loc[g, 'startup_cost'] * value(v[g,t]) 
                   for g in generators.index for t in range(24))
expected_gen_cost = value(prob.objective) - startup_costs

print(f"\nCost Breakdown:")
print(f"Startup costs: ${startup_costs:,.0f}")
print(f"Expected generation cost: ${expected_gen_cost:,.0f}")
print(f"Total expected cost: ${value(prob.objective):,.0f}")

## 6. Real-world Implementation Considerations

The optimization models we've implemented represent simplified versions of what runs in actual power system control rooms and markets. Real systems face several additional challenges:

**Computational Scale**: ISOs solve unit commitment for thousands of generators across multiple scenarios. The California ISO's market optimization involves over 6,000 generators and 30,000 constraints, solved every 5 minutes.

**Solution Quality vs Speed**: Market clearing must complete within strict time limits (typically 10-30 minutes for day-ahead markets). This requires careful tuning of solver parameters and acceptance of small optimality gaps.

**Approximations in Practice**: Full AC optimal power flow remains computationally intractable for large systems. Operators use various approximations including DC OPF with loss adjustments, linear sensitivity factors for security analysis, and convex relaxations for voltage constraints.

**Emerging Challenges**: The growth of distributed energy resources (DERs) is fundamentally changing optimization requirements. Future systems must coordinate millions of small resources, handle increased uncertainty, and operate distribution networks actively.

These models form the mathematical foundation that keeps the lights on reliably and economically. Understanding their formulation and limitations is essential for the next generation of power system engineers working to integrate renewable energy and maintain grid reliability.

## Summary

This lesson demonstrated how optimization techniques are applied to solve critical power system operational problems:

1. **Advanced Economic Dispatch**: Handling real-world complications like piecewise costs and prohibited zones
2. **Unit Commitment**: Mixed-integer programming for generator scheduling with complex technical constraints
3. **Security-Constrained Optimization**: Ensuring reliability through N-1 contingency analysis
4. **Optimal Power Flow**: Comparing DC and linearized AC formulations
5. **Renewable Integration**: Stochastic optimization for handling uncertainty

These optimization models run continuously in control rooms and markets worldwide, ensuring reliable and economical electricity delivery. As the grid evolves with more renewable resources and distributed generation, these techniques will become even more critical for maintaining system stability and efficiency.